In [6]:
# Importing necessary libraries and installing dependencies
%pip install pandas
%pip install altair

import pandas as pd
import altair as alt

# Enabling Altair's mimetype renderer for interactive visualization
alt.renderers.enable('mimetype')

# Reading the dataset
url = 'https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2021/2021-07-27/olympics.csv'
df = pd.read_csv(url)

# Data Preparation for the Heatmap
# Aggregating medal counts by Sport, Nation, and Season
medal_counts = (
    df[df['medal'].notna()]  # Exclude rows without medals
    .groupby(['sport', 'noc', 'season'])  # Group by Sport, Nation, and Season
    .size()
    .reset_index(name='medal_count')  # Count medals
)

# Adding a column to distinguish between Summer and Winter for coloring
medal_counts['season_color'] = medal_counts['season'].apply(
    lambda x: 'blue' if x == 'Summer' else 'orange'
)

# Creating the heatmap
heatmap = alt.Chart(medal_counts).mark_rect().encode(
    x=alt.X('sport:N', title='Sport', sort=None),
    y=alt.Y('noc:N', title='Nation', sort=None),
    color=alt.Color(
        'medal_count:Q',
        scale=alt.Scale(scheme='blueorange', domainMid=0),
        title='Number of Medals'
    ),
    tooltip=['sport', 'noc', 'season', 'medal_count']
).properties(
    width=1600,
    height=1200,
    title='Olympic Medals Heatmap: Sports vs Nations'
)

heatmap


Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting


In [19]:
import altair as alt
import pandas as pd
import json
import pycountry
import urllib.request
from vega_datasets import data

# Load the TopoJSON file
topojson_url = data.world_110m.url
with urllib.request.urlopen(topojson_url) as response:
    topojson = json.load(response)

# Inspect structure (if needed)
# print(json.dumps(topojson, indent=2))

# Extract mappings of numeric IDs and ISO3 codes
country_mappings = [
    {
        "id": feature["id"],
        "iso_alpha3": feature["properties"]["ISO_A3"]  # Adjust the key based on actual structure
    }
    for feature in topojson["objects"]["countries"]["geometries"]
]
countries = pd.DataFrame(country_mappings)

# Load the Olympic dataset
url = 'https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2021/2021-07-27/olympics.csv'
df = pd.read_csv(url)

# Function to map NOC to ISO3 using pycountry
def noc_to_iso3(noc):
    try:
        country = pycountry.countries.lookup(noc)
        return country.alpha_3
    except LookupError:
        return None

# Aggregate medal counts by nation
medal_counts_by_nation = (
    df[df['medal'].notna()]
    .groupby('noc')
    .size()
    .reset_index(name='medal_count')
)

# Manual mapping for missing NOC to ISO3 codes
manual_mapping = {
    'AHO': 'ANT',  # Netherlands Antilles
    'ALG': 'DZA',  # Algeria
    'ANZ': None,   # Australasia (no ISO3 code, not a country)
    'BOH': 'CZE',  # Bohemia (Czech Republic)
    'BWI': None,   # British West Indies (no ISO3 code)
    'YUG': 'SRB',  # Yugoslavia -> Serbia
    'SCG': 'SRB',  # Serbia and Montenegro -> Serbia
    'URS': 'RUS',  # Soviet Union -> Russia
    'TCH': 'CZE',  # Czechoslovakia -> Czech Republic
    'FRG': 'DEU',  # West Germany -> Germany
    'GDR': 'DEU',  # East Germany -> Germany
    # Add additional mappings if needed
}

# Function to handle both automatic and manual ISO3 mapping
def handle_missing_iso3(noc):
    if pd.isna(noc_to_iso3(noc)):
        return manual_mapping.get(noc, None)
    return noc_to_iso3(noc)

# Apply the function to map NOC to ISO3 codes
medal_counts_by_nation['iso_alpha3'] = medal_counts_by_nation['noc'].apply(handle_missing_iso3)

# Drop rows with missing ISO3 codes
medal_counts_by_nation = medal_counts_by_nation.dropna(subset=['iso_alpha3'])

# Merge medal data with numeric country IDs
medal_counts_with_ids = pd.merge(
    medal_counts_by_nation,
    countries,
    on='iso_alpha3',
    how='inner'
)

# Visualize the data on a world map
world = alt.topo_feature(data.world_110m.url, 'countries')

world_map = alt.Chart(world).mark_geoshape().encode(
    color=alt.Color(
        'medal_count:Q', 
        scale=alt.Scale(scheme='goldred'), 
        title='Number of Medals'
    ),
    tooltip=[
        alt.Tooltip('noc:N', title='NOC'),
        alt.Tooltip('medal_count:Q', title='Medal Count')
    ]
).transform_lookup(
    lookup='id',  # Use numeric country IDs for the lookup
    from_=alt.LookupData(medal_counts_with_ids, 'id', ['medal_count', 'noc'])
).properties(
    width=1000,
    height=600,
    title='Olympic Medals by Country'
).project(
    type='equirectangular'  # Equirectangular projection for better world map display
)

world_map


KeyError: 'properties'

In [25]:
import altair as alt
import pandas as pd
import json
import pycountry
import urllib.request

# Load the TopoJSON file from the provided URL
topojson_url = "https://vega.github.io/vega-datasets/data/world-110m.json"
with urllib.request.urlopen(topojson_url) as response:
    topojson = json.load(response)

# Extract mappings of numeric IDs and ISO3 codes
# Note: The 'id' in the TopoJSON file is numeric.
country_mappings = [
    {"id": int(feature["id"]), "iso_alpha3": str(feature["id"])}
    for feature in topojson["objects"]["countries"]["geometries"]
    if "id" in feature
]
countries = pd.DataFrame(country_mappings)

# Function to map NOC to ISO3 using pycountry
def noc_to_iso3(noc):
    try:
        country = pycountry.countries.lookup(noc)
        return country.alpha_3
    except LookupError:
        return None

# Load the Olympic dataset
url = "https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2021/2021-07-27/olympics.csv"
df = pd.read_csv(url)

# Aggregate medal counts by nation
medal_counts_by_nation = (
    df[df["medal"].notna()]
    .groupby("noc")
    .size()
    .reset_index(name="medal_count")
)

# Manual mapping for missing NOC to ISO3 codes
manual_mapping = {
    "AHO": "ANT",  # Netherlands Antilles
    "ALG": "DZA",  # Algeria
    "ANZ": None,  # Australasia (no ISO3 code, not a country)
    "BOH": "CZE",  # Bohemia (Czech Republic)
    "BWI": None,  # British West Indies (no ISO3 code)
    "YUG": "SRB",  # Yugoslavia -> Serbia
    "SCG": "SRB",  # Serbia and Montenegro -> Serbia
    "URS": "RUS",  # Soviet Union -> Russia
    "TCH": "CZE",  # Czechoslovakia -> Czech Republic
    "FRG": "DEU",  # West Germany -> Germany
    "GDR": "DEU",  # East Germany -> Germany
    # Add additional mappings if needed
}

# Function to handle both automatic and manual ISO3 mapping
def handle_missing_iso3(noc):
    if pd.isna(noc_to_iso3(noc)):
        return manual_mapping.get(noc, None)
    return noc_to_iso3(noc)

# Apply the function to map NOC to ISO3 codes
medal_counts_by_nation["iso_alpha3"] = medal_counts_by_nation["noc"].apply(handle_missing_iso3)

# Convert iso_alpha3 column to string for compatibility
medal_counts_by_nation["iso_alpha3"] = medal_counts_by_nation["iso_alpha3"].astype(str)
countries["iso_alpha3"] = countries["iso_alpha3"].astype(str)

# Drop rows with missing ISO3 codes
medal_counts_by_nation = medal_counts_by_nation.dropna(subset=["iso_alpha3"])

# Merge medal data with numeric country IDs
medal_counts_with_ids = pd.merge(
    medal_counts_by_nation,
    countries,
    on="iso_alpha3",
    how="inner"
)

# Visualize the data on a world map
world = alt.topo_feature(topojson_url, "countries")

world_map = alt.Chart(world).mark_geoshape().encode(
    color=alt.Color(
        "medal_count:Q",
        scale=alt.Scale(scheme="goldred"),
        title="Number of Medals"
    ),
    tooltip=[
        alt.Tooltip("noc:N", title="NOC"),
        alt.Tooltip("medal_count:Q", title="Medal Count")
    ]
).transform_lookup(
    lookup="id",  # Use numeric country IDs for the lookup
    from_=alt.LookupData(medal_counts_with_ids, "id", ["medal_count", "noc"])
).properties(
    width=1000,
    height=600,
    title="Olympic Medals by Country"
).project(
    type="equirectangular"  # Equirectangular projection for better world map display
)

world_map
world_map.save('olympic_medals_map.html')

print("Unique IDs in TopoJSON:", countries["id"].unique())
print("Unique IDs in Medal Data:", medal_counts_with_ids["id"].unique())
print(medal_counts_with_ids.head())
print(world)

print("Unique iso_alpha3 in Medal Data:", medal_counts_by_nation["iso_alpha3"].unique())
print("Unique iso_alpha3 in Countries Data:", countries["iso_alpha3"].unique())

print("Type of iso_alpha3 in Medal Data:", medal_counts_by_nation["iso_alpha3"].dtype)
print("Type of iso_alpha3 in Countries Data:", countries["iso_alpha3"].dtype)



Unique IDs in TopoJSON: [  4  24   8 784  32  51  10 260  36  40  31 108  56 204 854  50 100  44
  70 112  84  68  76  96  64  72 140 124 756 152 156 384 120 180 178 170
 188 192 -99 196 203 276 262 208 214  12 218 818 232 724 233 231 246 242
 238 250 266 826 268 288 324 270 624 226 300 304 320 328 340 191 332 348
 360 356 372 364 368 352 376 380 388 400 392 398 404 417 116 410 414 418
 422 430 434 144 426 440 442 428 504 498 450 484 807 466 104 499 496 508
 478 454 458 516 540 562 566 558 528 578 524 554 512 586 591 604 608 598
 616 630 408 620 600 275 634 642 643 646 732 682 729 728 686  90 694 222
 706 688 740 703 705 752 748 760 148 768 764 762 795 626 780 788 792 158
 834 800 804 858 840 860 862 704 548 887 710 894 716]
Unique IDs in Medal Data: []
Empty DataFrame
Columns: [noc, medal_count, iso_alpha3, id]
Index: []
UrlData({
  format: TopoDataFormat({
    feature: 'countries',
    type: 'topojson'
  }),
  url: 'https://vega.github.io/vega-datasets/data/world-110m.json'
})
Unique

In [27]:
import altair as alt
import pandas as pd
import json
import pycountry
import urllib.request

# Load the TopoJSON file from the provided URL
topojson_url = "https://vega.github.io/vega-datasets/data/world-110m.json"
with urllib.request.urlopen(topojson_url) as response:
    topojson = json.load(response)

# Extract mappings of numeric IDs and ISO3 codes
country_mappings = [
    {"id": int(feature["id"]), "iso_alpha3": str(feature["id"])}
    for feature in topojson["objects"]["countries"]["geometries"]
    if "id" in feature
]
countries = pd.DataFrame(country_mappings)

# Function to map NOC to ISO3 using pycountry
def noc_to_iso3(noc):
    try:
        country = pycountry.countries.lookup(noc)
        return country.alpha_3
    except LookupError:
        return None

# Load the Olympic dataset
url = "https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2021/2021-07-27/olympics.csv"
df = pd.read_csv(url)

# Aggregate medal counts by nation
medal_counts_by_nation = (
    df[df["medal"].notna()]
    .groupby("noc")
    .size()
    .reset_index(name="medal_count")
)

# Manual mapping for missing NOC to ISO3 codes
manual_mapping = {
    "AHO": "ANT",  # Netherlands Antilles
    "ALG": "DZA",  # Algeria
    "ANZ": None,  # Australasia (no ISO3 code, not a country)
    "BOH": "CZE",  # Bohemia (Czech Republic)
    "BWI": None,  # British West Indies (no ISO3 code)
    "YUG": "SRB",  # Yugoslavia -> Serbia
    "SCG": "SRB",  # Serbia and Montenegro -> Serbia
    "URS": "RUS",  # Soviet Union -> Russia
    "TCH": "CZE",  # Czechoslovakia -> Czech Republic
    "FRG": "DEU",  # West Germany -> Germany
    "GDR": "DEU",  # East Germany -> Germany
    # Add additional mappings if needed
}

# Function to handle both automatic and manual ISO3 mapping
def handle_missing_iso3(noc):
    if pd.isna(noc_to_iso3(noc)):
        return manual_mapping.get(noc, None)
    return noc_to_iso3(noc)

# Apply the function to map NOC to ISO3 codes
medal_counts_by_nation["iso_alpha3"] = medal_counts_by_nation["noc"].apply(handle_missing_iso3)

# Convert iso_alpha3 columns to uppercase for consistency
medal_counts_by_nation["iso_alpha3"] = medal_counts_by_nation["iso_alpha3"].str.upper()
countries["iso_alpha3"] = countries["iso_alpha3"].str.upper()

# Filter out invalid iso_alpha3 values
valid_iso_alpha3 = countries["iso_alpha3"].unique()
medal_counts_by_nation = medal_counts_by_nation[
    medal_counts_by_nation["iso_alpha3"].isin(valid_iso_alpha3)
]

# Merge medal data with numeric country IDs
medal_counts_with_ids = pd.merge(
    medal_counts_by_nation,
    countries,
    on="iso_alpha3",
    how="inner"
)

# Debugging step: Check merged data
print("Merged Data Sample:")
print(medal_counts_with_ids.head())

# Visualize the data on a world map
world = alt.topo_feature(topojson_url, "countries")

world_map = alt.Chart(world).mark_geoshape().encode(
    color=alt.Color(
        "medal_count:Q",
        scale=alt.Scale(scheme="goldred"),
        title="Number of Medals"
    ),
    tooltip=[
        alt.Tooltip("noc:N", title="NOC"),
        alt.Tooltip("medal_count:Q", title="Medal Count")
    ]
).transform_lookup(
    lookup="id",  # Use numeric country IDs for the lookup
    from_=alt.LookupData(medal_counts_with_ids, "id", ["medal_count", "noc"])
).properties(
    width=1000,
    height=600,
    title="Olympic Medals by Country"
).project(
    type="equirectangular"  # Equirectangular projection for better world map display
)

world_map


Merged Data Sample:
Empty DataFrame
Columns: [noc, medal_count, iso_alpha3, id]
Index: []


<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting


In [29]:
import altair as alt
import pandas as pd
import json
import pycountry
import urllib.request

# Load the TopoJSON file from the provided URL
topojson_url = "https://vega.github.io/vega-datasets/data/world-110m.json"
with urllib.request.urlopen(topojson_url) as response:
    topojson = json.load(response)

# Extract mappings of numeric IDs and ISO3 codes
country_mappings = [
    {"id": int(feature["id"]), "iso_alpha3": str(feature["id"])}
    for feature in topojson["objects"]["countries"]["geometries"]
    if "id" in feature
]
countries = pd.DataFrame(country_mappings)

# Function to map NOC to ISO3 using pycountry
def noc_to_iso3(noc):
    try:
        country = pycountry.countries.lookup(noc)
        return country.alpha_3
    except LookupError:
        return None

# Load the Olympic dataset
url = "https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2021/2021-07-27/olympics.csv"
df = pd.read_csv(url)

# Aggregate medal counts by nation
medal_counts_by_nation = (
    df[df["medal"].notna()]
    .groupby("noc")
    .size()
    .reset_index(name="medal_count")
)

# Manual mapping for missing NOC to ISO3 codes
manual_mapping = {
    "AHO": "ANT",  # Netherlands Antilles
    "ALG": "DZA",  # Algeria
    "ANZ": None,  # Australasia (no ISO3 code, not a country)
    "BOH": "CZE",  # Bohemia (Czech Republic)
    "BWI": None,  # British West Indies (no ISO3 code)
    "YUG": "SRB",  # Yugoslavia -> Serbia
    "SCG": "SRB",  # Serbia and Montenegro -> Serbia
    "URS": "RUS",  # Soviet Union -> Russia
    "TCH": "CZE",  # Czechoslovakia -> Czech Republic
    "FRG": "DEU",  # West Germany -> Germany
    "GDR": "DEU",  # East Germany -> Germany
    # Add additional mappings if needed
}

# Function to handle both automatic and manual ISO3 mapping
def handle_missing_iso3(noc):
    if pd.isna(noc_to_iso3(noc)):
        return manual_mapping.get(noc, None)
    return noc_to_iso3(noc)

# Apply the function to map NOC to ISO3 codes
medal_counts_by_nation["iso_alpha3"] = medal_counts_by_nation["noc"].apply(handle_missing_iso3)

# Convert iso_alpha3 columns to uppercase for consistency
medal_counts_by_nation["iso_alpha3"] = medal_counts_by_nation["iso_alpha3"].str.upper()
countries["iso_alpha3"] = countries["iso_alpha3"].str.upper()

# Filter out invalid iso_alpha3 values
valid_iso_alpha3 = countries["iso_alpha3"].unique()
medal_counts_by_nation = medal_counts_by_nation[
    medal_counts_by_nation["iso_alpha3"].isin(valid_iso_alpha3)
]

# Merge medal data with numeric country IDs
medal_counts_with_ids = pd.merge(
    medal_counts_by_nation,
    countries,
    on="iso_alpha3",
    how="inner"
)

# Debugging step: Check merged data
print("Merged Data Sample:")
print(medal_counts_with_ids.head())

# Visualize the data on a world map
world = alt.topo_feature(topojson_url, "countries")

world_map = alt.Chart(world).mark_geoshape().encode(
    color=alt.Color(
        "medal_count:Q",
        scale=alt.Scale(scheme="goldred"),
        title="Number of Medals"
    ),
    tooltip=[
        alt.Tooltip("noc:N", title="NOC"),
        alt.Tooltip("medal_count:Q", title="Medal Count")
    ]
).transform_lookup(
    lookup="id",  # Use numeric country IDs for the lookup
    from_=alt.LookupData(medal_counts_with_ids, "id", ["medal_count", "noc"])
).properties(
    width=1000,
    height=600,
    title="Olympic Medals by Country"
).project(
    type="equirectangular"  # Equirectangular projection for better world map display
)

world_map

# Check unique iso_alpha3 in each dataset
print("Unique iso_alpha3 in Medal Data:", medal_counts_by_nation["iso_alpha3"].unique())
print("Unique iso_alpha3 in Countries Data:", countries["iso_alpha3"].unique())

# Check for intersection
overlap = set(medal_counts_by_nation["iso_alpha3"]).intersection(set(countries["iso_alpha3"]))
print("Overlap:", overlap)



Merged Data Sample:
Empty DataFrame
Columns: [noc, medal_count, iso_alpha3, id]
Index: []
Unique iso_alpha3 in Medal Data: []
Unique iso_alpha3 in Countries Data: ['4' '24' '8' '784' '32' '51' '10' '260' '36' '40' '31' '108' '56' '204'
 '854' '50' '100' '44' '70' '112' '84' '68' '76' '96' '64' '72' '140'
 '124' '756' '152' '156' '384' '120' '180' '178' '170' '188' '192' '-99'
 '196' '203' '276' '262' '208' '214' '12' '218' '818' '232' '724' '233'
 '231' '246' '242' '238' '250' '266' '826' '268' '288' '324' '270' '624'
 '226' '300' '304' '320' '328' '340' '191' '332' '348' '360' '356' '372'
 '364' '368' '352' '376' '380' '388' '400' '392' '398' '404' '417' '116'
 '410' '414' '418' '422' '430' '434' '144' '426' '440' '442' '428' '504'
 '498' '450' '484' '807' '466' '104' '499' '496' '508' '478' '454' '458'
 '516' '540' '562' '566' '558' '528' '578' '524' '554' '512' '586' '591'
 '604' '608' '598' '616' '630' '408' '620' '600' '275' '634' '642' '643'
 '646' '732' '682' '729' '728' '686' '

In [30]:
import altair as alt
import pandas as pd
import json
import pycountry
import urllib.request

# Load the TopoJSON file from the provided URL
topojson_url = "https://vega.github.io/vega-datasets/data/world-110m.json"
with urllib.request.urlopen(topojson_url) as response:
    topojson = json.load(response)

# Extract mappings of numeric IDs
country_mappings = [{"id": int(feature["id"])} for feature in topojson["objects"]["countries"]["geometries"]]
countries = pd.DataFrame(country_mappings)

# Manual mapping between numeric IDs and ISO3 codes
id_to_iso3_mapping = {
    4: "AFG",  # Afghanistan
    8: "ALB",  # Albania
    12: "DZA",  # Algeria
    24: "AGO",  # Angola
    32: "ARG",  # Argentina
    36: "AUS",  # Australia
    40: "AUT",  # Austria
    51: "ARM",  # Armenia
    56: "BEL",  # Belgium
    70: "BIH",  # Bosnia and Herzegovina
    100: "BGR",  # Bulgaria
    124: "CAN",  # Canada
    152: "CHL",  # Chile
    156: "CHN",  # China
    170: "COL",  # Colombia
    191: "HRV",  # Croatia
    203: "CZE",  # Czech Republic
    208: "DNK",  # Denmark
    214: "DOM",  # Dominican Republic
    218: "ECU",  # Ecuador
    233: "EST",  # Estonia
    246: "FIN",  # Finland
    250: "FRA",  # France
    276: "DEU",  # Germany
    300: "GRC",  # Greece
    348: "HUN",  # Hungary
    352: "ISL",  # Iceland
    356: "IND",  # India
    360: "IDN",  # Indonesia
    364: "IRN",  # Iran
    372: "IRL",  # Ireland
    376: "ISR",  # Israel
    380: "ITA",  # Italy
    392: "JPN",  # Japan
    398: "KAZ",  # Kazakhstan
    410: "KOR",  # South Korea
    428: "LVA",  # Latvia
    440: "LTU",  # Lithuania
    442: "LUX",  # Luxembourg
    450: "MDG",  # Madagascar
    484: "MEX",  # Mexico
    499: "MNE",  # Montenegro
    504: "MAR",  # Morocco
    528: "NLD",  # Netherlands
    554: "NZL",  # New Zealand
    566: "NGA",  # Nigeria
    578: "NOR",  # Norway
    586: "PAK",  # Pakistan
    616: "POL",  # Poland
    620: "PRT",  # Portugal
    642: "ROU",  # Romania
    643: "RUS",  # Russia
    703: "SVK",  # Slovakia
    705: "SVN",  # Slovenia
    710: "ZAF",  # South Africa
    724: "ESP",  # Spain
    752: "SWE",  # Sweden
    756: "CHE",  # Switzerland
    826: "GBR",  # United Kingdom
    840: "USA",  # United States
    860: "UZB",  # Uzbekistan
    894: "ZMB",  # Zambia
    784: "ARE",  # United Arab Emirates
    10: "ATA",   # Antarctica (can be skipped if irrelevant)
    260: "MYT",  # Mayotte
    31: "AZE",   # Azerbaijan
    108: "BDI",  # Burundi
    204: "BEN",  # Benin
    854: "BFA",  # Burkina Faso
    50: "BGD",   # Bangladesh
    44: "BHS",   # Bahamas
    112: "BLR",  # Belarus
    84: "BLZ",   # Belize
    68: "BOL",   # Bolivia
    76: "BRA",   # Brazil
    96: "BRN",   # Brunei
    64: "BTN",   # Bhutan
    72: "BWA",   # Botswana
    140: "CAF",  # Central African Republic
    384: "CIV",  # Côte d'Ivoire
    120: "CMR",  # Cameroon
    180: "COD",  # Congo (Kinshasa)
    178: "COG",  # Congo (Brazzaville)
    188: "CRI",  # Costa Rica
    192: "CUB",  # Cuba
    -99: None,   # Unknown or undefined
    196: "CYP",  # Cyprus
    262: "DJI",  # Djibouti
    818: "EGY",  # Egypt
    232: "ERI",  # Eritrea
    231: "ETH",  # Ethiopia
    242: "FJI",  # Fiji
    238: "FLK",  # Falkland Islands (skipped if irrelevant)
    266: "GAB",  # Gabon
    268: "GEO",  # Georgia
    288: "GHA",  # Ghana
    324: "GIN",  # Guinea
    270: "GMB",  # Gambia
    624: "GNB",  # Guinea-Bissau
    226: "GNQ",  # Equatorial Guinea
    304: "GRL",  # Greenland
    320: "GTM",  # Guatemala
    328: "GUY",  # Guyana
    340: "HND",  # Honduras
    332: "HTI",  # Haiti
    368: "IRQ",  # Iraq
    388: "JAM",  # Jamaica
    400: "JOR",  # Jordan
    404: "KEN",  # Kenya
    417: "KGZ",  # Kyrgyzstan
    116: "KHM",  # Cambodia
    414: "KWT",  # Kuwait
    418: "LAO",  # Laos
    422: "LBN",  # Lebanon
    430: "LBR",  # Liberia
    434: "LBY",  # Libya
    144: "LKA",  # Sri Lanka
    426: "LSO",  # Lesotho
    498: "MDA",  # Moldova
    807: "MKD",  # North Macedonia
    466: "MLI",  # Mali
    104: "MMR",  # Myanmar
    496: "MNG",  # Mongolia
    508: "MOZ",  # Mozambique
    478: "MRT",  # Mauritania
    454: "MWI",  # Malawi
    458: "MYS",  # Malaysia
    516: "NAM",  # Namibia
    540: "NCL",  # New Caledonia
    562: "NER",  # Niger
    558: "NIC",  # Nicaragua
    524: "NPL",  # Nepal
    512: "OMN",  # Oman
    591: "PAN",  # Panama
    604: "PER",  # Peru
    608: "PHL",  # Philippines
    598: "PNG",  # Papua New Guinea
    630: "PRI",  # Puerto Rico
    408: "PRK",  # North Korea
    600: "PRY",  # Paraguay
    275: "PSE",  # Palestine
    634: "QAT",  # Qatar
    646: "RWA",  # Rwanda
    732: "ESH",  # Western Sahara
    682: "SAU",  # Saudi Arabia
    729: "SDN",  # Sudan
    728: "SSD",  # South Sudan
    686: "SEN",  # Senegal
    90: "SLB",   # Solomon Islands
    694: "SLE",  # Sierra Leone
    222: "SLV",  # El Salvador
    706: "SOM",  # Somalia
    688: "SRB",  # Serbia
    740: "SUR",  # Suriname
    748: "SWZ",  # Eswatini (Swaziland)
    760: "SYR",  # Syria
    148: "TCD",  # Chad
    768: "TGO",  # Togo
    764: "THA",  # Thailand
    762: "TJK",  # Tajikistan
    795: "TKM",  # Turkmenistan
    626: "TLS",  # Timor-Leste
    780: "TTO",  # Trinidad and Tobago
    788: "TUN",  # Tunisia
    792: "TUR",  # Turkey
    158: "TWN",  # Taiwan
    834: "TZA",  # Tanzania
    800: "UGA",  # Uganda
    804: "UKR",  # Ukraine
    858: "URY",  # Uruguay
    862: "VEN",  # Venezuela
    704: "VNM",  # Vietnam
    548: "VUT",  # Vanuatu
    887: "YEM",  # Yemen
    716: "ZWE",  # Zimbabwe
}
manual_iso3_mapping = {
    "AHO": "ANT",  # Netherlands Antilles
    "ALG": "DZA",  # Algeria
    "ANZ": "AUS",  # Australasia
    "BAH": "BHS",  # Bahamas
    "BAR": "BRB",  # Barbados
    "BER": "BMU",  # Bermuda
    "BOH": "CZE",  # Bohemia -> Czech Republic
    "BOT": "BWA",  # Botswana
    "BUL": "BGR",  # Bulgaria
    "CHI": "CHL",  # Chile
    "CRC": "CRI",  # Costa Rica
    "CRO": "HRV",  # Croatia
    "DEN": "DNK",  # Denmark
    "EUN": "RUS",  # Unified Team → Russia
    "FIJ": "FJI",  # Fiji
    "FRG": "DEU",  # West Germany → Germany
    "GDR": "DEU",  # East Germany → Germany
    "GER": "DEU",  # Germany
    "GRE": "GRC",  # Greece
    "GRN": "GRD",  # Grenada
    "GUA": "GTM",  # Guatemala
    "HAI": "HTI",  # Haiti
    "HKG": "HKG",  # Hong Kong
    "INA": "IDN",  # Indonesia
    "IOA": None,   # Independent Olympic Athletes → No ISO3 equivalent
    "IRI": "IRN",  # Iran
    "ISV": "VIR",  # U.S. Virgin Islands
    "KOS": "XKX",  # Kosovo
    "KSA": "SAU",  # Saudi Arabia
    "KUW": "KWT",  # Kuwait
    "LAT": "LVA",  # Latvia
    "LIB": "LBN",  # Lebanon
    "LIE": "LIE",  # Liechtenstein
    "MAS": "MYS",  # Malaysia
    "MGL": "MNG",  # Mongolia
    "MON": "MCO",  # Monaco
    "MRI": "MUS",  # Mauritius
    "NED": "NLD",  # Netherlands
    "NEP": "NPL",  # Nepal
    "NGR": "NGA",  # Nigeria
    "NIG": "NER",  # Niger
    "PAR": "PRY",  # Paraguay
    "PHI": "PHL",  # Philippines
    "POR": "PRT",  # Portugal
    "PUR": "PRI",  # Puerto Rico
    "RSA": "ZAF",  # South Africa
    "SCG": "SRB",  # Serbia and Montenegro
    "SGP": "SGP",  # Singapore
    "SLO": "SVN",  # Slovenia
    "SRI": "LKA",  # Sri Lanka
    "SUD": "SDN",  # Sudan
    "SUI": "CHE",  # Switzerland
    "TAN": "TZA",  # Tanzania
    "TCH": "CZE",  # Czechoslovakia → Czech Republic
    "TGA": "TON",  # Tonga
    "TOG": "TGO",  # Togo
    "TPE": "TWN",  # Chinese Taipei → Taiwan
    "UAE": "ARE",  # United Arab Emirates
    "UAR": "EGY",  # United Arab Republic → Egypt
    "URS": "RUS",  # Soviet Union → Russia
    "URU": "URY",  # Uruguay
    "VIE": "VNM",  # Vietnam
    "WIF": None,   # West Indies Federation → No ISO3 equivalent
    "YUG": "SRB",  # Yugoslavia → Serbia
    "ZAM": "ZMB",  # Zambia
    "ZIM": "ZWE",  # Zimbabwe
}
# Apply manual mappings to medal_counts_by_nation
medal_counts_by_nation["iso_alpha3"] = medal_counts_by_nation["iso_alpha3"].replace(manual_iso3_mapping)

# Add ISO3 codes to countries DataFrame
countries["iso_alpha3"] = countries["id"].map(id_to_iso3_mapping)
# Filter out any remaining invalid ISO3 codes
medal_counts_by_nation = medal_counts_by_nation[medal_counts_by_nation["iso_alpha3"].notnull()]






# Load the Olympic dataset
url = "https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2021/2021-07-27/olympics.csv"
df = pd.read_csv(url)

# Aggregate medal counts by nation
medal_counts_by_nation = (
    df[df["medal"].notna()]
    .groupby("noc")
    .size()
    .reset_index(name="medal_count")
)

# Convert iso_alpha3 columns to uppercase for consistency
medal_counts_by_nation["iso_alpha3"] = medal_counts_by_nation["noc"].str.upper()
countries["iso_alpha3"] = countries["iso_alpha3"].str.upper()

# Merge medal data with numeric country IDs
medal_counts_with_ids = pd.merge(
    medal_counts_by_nation,
    countries,
    on="iso_alpha3",
    how="inner"
)

# Debugging step: Check merged data
print("Merged Data Sample:")
print(medal_counts_with_ids.head())

# Visualize the data on a world map
world = alt.topo_feature(topojson_url, "countries")

world_map = alt.Chart(world).mark_geoshape().encode(
    color=alt.Color(
        "medal_count:Q",
        scale=alt.Scale(scheme="goldred"),
        title="Number of Medals"
    ),
    tooltip=[
        alt.Tooltip("noc:N", title="NOC"),
        alt.Tooltip("medal_count:Q", title="Medal Count")
    ]
).transform_lookup(
    lookup="id",  # Use numeric country IDs for the lookup
    from_=alt.LookupData(medal_counts_with_ids, "id", ["medal_count", "noc"])
).properties(
    width=1000,
    height=600,
    title="Olympic Medals by Country"
).project(
    type="equirectangular"  # Equirectangular projection for better world map display
)
remaining_missing_nations = set(medal_counts_by_nation["noc"]) - set(medal_counts_with_ids["noc"])
print("Remaining Missing Nations (NOC Codes):", remaining_missing_nations)

world_map






Merged Data Sample:
   noc  medal_count iso_alpha3  id
0  AFG            2        AFG   4
1  ARG          274        ARG  32
2  ARM           16        ARM  51
3  AUS         1320        AUS  36
4  AUT          450        AUT  40
Remaining Missing Nations (NOC Codes): {'SCG', 'HKG', 'IOA', 'NEP', 'SUI', 'SGP', 'VIE', 'BER', 'MAS', 'GDR', 'SRI', 'AHO', 'SLO', 'LAT', 'ZIM', 'NED', 'BAR', 'GUA', 'TGA', 'MGL', 'URS', 'UAR', 'MRI', 'SUD', 'URU', 'LIB', 'BOH', 'PAR', 'HAI', 'FIJ', 'FRG', 'ALG', 'BUL', 'ANZ', 'PHI', 'CHI', 'CRO', 'INA', 'BAH', 'IRI', 'MON', 'GRN', 'ZAM', 'NGR', 'LIE', 'KSA', 'POR', 'PUR', 'TPE', 'WIF', 'KOS', 'BOT', 'DEN', 'TCH', 'GER', 'UAE', 'ISV', 'TOG', 'YUG', 'KUW', 'NIG', 'TAN', 'RSA', 'EUN', 'CRC', 'GRE'}


alt.Chart(...)

In [31]:
import altair as alt
import pandas as pd
import json
import urllib.request

# Load the TopoJSON file from the provided URL
topojson_url = "https://vega.github.io/vega-datasets/data/world-110m.json"
with urllib.request.urlopen(topojson_url) as response:
    topojson = json.load(response)

# Load the Olympic dataset
url = "https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2021/2021-07-27/olympics.csv"
df = pd.read_csv(url)

# Aggregate medal counts by nation
medal_counts_by_nation = (
    df[df["medal"].notna()]
    .groupby(["noc", "season"])
    .size()
    .reset_index(name="medal_count")
)

# Manual mapping between numeric IDs and ISO3 codes
id_to_iso3_mapping = {
    4: "AFG", 8: "ALB", 12: "DZA", 24: "AGO", 32: "ARG", 36: "AUS", 40: "AUT", 51: "ARM",
    56: "BEL", 70: "BIH", 100: "BGR", 124: "CAN", 152: "CHL", 156: "CHN", 170: "COL",
    191: "HRV", 203: "CZE", 208: "DNK", 214: "DOM", 218: "ECU", 233: "EST", 246: "FIN",
    250: "FRA", 276: "DEU", 300: "GRC", 348: "HUN", 352: "ISL", 356: "IND", 360: "IDN",
    364: "IRN", 372: "IRL", 376: "ISR", 380: "ITA", 392: "JPN", 398: "KAZ", 410: "KOR",
    428: "LVA", 440: "LTU", 442: "LUX", 450: "MDG", 484: "MEX", 499: "MNE", 504: "MAR",
    528: "NLD", 554: "NZL", 566: "NGA", 578: "NOR", 586: "PAK", 616: "POL", 620: "PRT",
    642: "ROU", 643: "RUS", 703: "SVK", 705: "SVN", 710: "ZAF", 724: "ESP", 752: "SWE",
    756: "CHE", 826: "GBR", 840: "USA", 860: "UZB", 894: "ZMB"
}

# Prepare country mapping DataFrame
country_mappings = [{"id": int(feature["id"])} for feature in topojson["objects"]["countries"]["geometries"]]
countries = pd.DataFrame(country_mappings)
countries["iso_alpha3"] = countries["id"].map(id_to_iso3_mapping)

# Merge medal data with numeric country IDs
medal_counts_by_nation["iso_alpha3"] = medal_counts_by_nation["noc"]
medal_counts_with_ids = pd.merge(
    medal_counts_by_nation,
    countries,
    on="iso_alpha3",
    how="inner"
)

# Create a parameter for selecting the season
season_selector = alt.param(
    name="season_select",
    bind=alt.binding_radio(options=["Summer", "Winter"], name="Season: "),
    value="Summer"
)

# Define the bar chart
bar_chart = alt.Chart(medal_counts_with_ids).mark_bar().encode(
    y=alt.Y("noc:N", axis=alt.Axis(title=None)),
    x=alt.X("sum(medal_count):Q", title="Number of Medals"),
    color=alt.Color(
        "season:N",
        scale=alt.Scale(domain=["Summer", "Winter"], range=["orange", "blue"]),
    ),
    fillOpacity=alt.condition(season_selector, alt.value(1), alt.value(0.3))
).transform_filter(season_selector).add_params(season_selector).properties(
    width=100,
    height=505
)

# Define the map chart
world = alt.topo_feature(topojson_url, "countries")

map_chart = alt.Chart(world).mark_geoshape().encode(
    color=alt.Color(
        "sum(medal_count):Q",
        scale=alt.Scale(domain=[0, 10, 100], scheme="goldred"),
        title="Number of Medals"
    ),
    tooltip=[
        alt.Tooltip("iso_alpha3:N", title="Country"),
        alt.Tooltip("sum(medal_count):Q", title="Medal Count")
    ]
).transform_lookup(
    lookup="id",
    from_=alt.LookupData(medal_counts_with_ids, "id", ["medal_count", "noc", "season"])
).transform_filter(season_selector).properties(
    width=400,
    height=350,
    title="Olympic Medals Map"
).project("equirectangular")

# Combine the bar chart and map chart
final_chart = alt.hconcat(bar_chart, map_chart).configure_legend(
    titleFontSize=15,
    labelFontSize=13
).properties(
    title=alt.TitleParams(
        text="Olympic Medals Explorer", anchor="middle", fontSize=20, fontWeight=800
    )
)

final_chart


alt.HConcatChart(...)

In [35]:
import altair as alt
import pandas as pd
import json
import urllib.request

# Load the TopoJSON file from the provided URL
topojson_url = "https://vega.github.io/vega-datasets/data/world-110m.json"
with urllib.request.urlopen(topojson_url) as response:
    topojson = json.load(response)

# Load the Olympic dataset
url = "https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2021/2021-07-27/olympics.csv"
df = pd.read_csv(url)

# Aggregate medal counts by nation and season
medal_counts_by_nation = (
    df[df["medal"].notna()]
    .groupby(["noc", "season"])
    .size()
    .reset_index(name="medal_count")
)

# Mapping between numeric IDs and ISO3 codes
id_to_iso3_mapping = {
    4: "AFG", 8: "ALB", 12: "DZA", 24: "AGO", 32: "ARG", 36: "AUS", 40: "AUT",
    51: "ARM", 56: "BEL", 70: "BIH", 100: "BGR", 124: "CAN", 152: "CHL",
    156: "CHN", 170: "COL", 191: "HRV", 203: "CZE", 208: "DNK", 214: "DOM",
    218: "ECU", 233: "EST", 246: "FIN", 250: "FRA", 276: "DEU", 300: "GRC",
    348: "HUN", 352: "ISL", 356: "IND", 360: "IDN", 364: "IRN", 372: "IRL",
    376: "ISR", 380: "ITA", 392: "JPN", 398: "KAZ", 410: "KOR", 428: "LVA",
    440: "LTU", 442: "LUX", 450: "MDG", 484: "MEX", 499: "MNE", 504: "MAR",
    528: "NLD", 554: "NZL", 566: "NGA", 578: "NOR", 586: "PAK", 616: "POL",
    620: "PRT", 642: "ROU", 643: "RUS", 703: "SVK", 705: "SVN", 710: "ZAF",
    724: "ESP", 752: "SWE", 756: "CHE", 826: "GBR", 840: "USA", 860: "UZB",
    894: "ZMB", 784: "ARE"
}

# Add numeric IDs to the medal dataset
medal_counts_by_nation["iso_alpha3"] = medal_counts_by_nation["noc"].str.upper()
countries = pd.DataFrame.from_dict(id_to_iso3_mapping, orient='index', columns=["iso_alpha3"]).reset_index().rename(columns={"index": "id"})

medal_counts_with_ids = pd.merge(
    medal_counts_by_nation,
    countries,
    on="iso_alpha3",
    how="inner"
)

# Load world map
world = alt.topo_feature(topojson_url, "countries")

# Create a parameter for selecting the season (Summer/Winter)
season_selector = alt.param(
    name="season_select",
    bind=alt.binding_radio(name="Season", options=["Summer", "Winter"]),
    value="Summer"  # Default selection
)

# Create the map chart with a season filter
world_map = alt.Chart(world).mark_geoshape().encode(
    color=alt.Color(
        "medal_count:Q",
        scale=alt.Scale(
            domain=[0, medal_counts_with_ids["medal_count"].max()],
            range=["lightblue", "darkblue"]
        ),
        title="Number of Medals"
    ),
    tooltip=[
        alt.Tooltip("noc:N", title="NOC"),
        alt.Tooltip("medal_count:Q", title="Medal Count"),
        alt.Tooltip("season:N", title="Season")
    ]
).transform_lookup(
    lookup="id",
    from_=alt.LookupData(medal_counts_with_ids, "id", ["medal_count", "noc", "season"])
).transform_filter(
    alt.datum.season == season_selector  # Filter for the selected season
).properties(
    width=800,
    height=400,
    title="Olympic Medals by Country (Switchable by Season)"
).project(
    type="equirectangular"
).add_params(
    season_selector
)

world_map


alt.Chart(...)

In [50]:
import altair as alt
import pandas as pd
import json
import urllib.request

# Load the dataset
data_path = "data/Updated_Medals_Summary_with_Numeric_ID.csv"
df = pd.read_csv(data_path)

# Load the TopoJSON file for the world map
topojson_url = "https://vega.github.io/vega-datasets/data/world-110m.json"
with urllib.request.urlopen(topojson_url) as response:
    topojson = json.load(response)

# Prepare the TopoJSON for use in Altair
world = alt.topo_feature(topojson_url, "countries")

# Create a parameter to toggle between Summer and Winter Olympics
season_param = alt.param(
    name="Season",
    bind=alt.binding_radio(options=["Summer", "Winter"], name="Season: "),
    value="Summer"  # Default value
)

# Filter the data to include only relevant columns
df_filtered = df[["noc", "numeric_id", "season", "Total_Medals"]]

# Create the visualization
world_map = alt.Chart(world).mark_geoshape().encode(
    color=alt.Color(
        "Total_Medals:Q",
        scale=alt.Scale(scheme="goldred"),
        title="Total Medals"
    ),
    tooltip=[
        alt.Tooltip("noc:N", title="NOC"),
        alt.Tooltip("season:N", title="Season"),
        alt.Tooltip("Total_Medals:Q", title="Total Medals")
    ]
).transform_lookup(
    lookup="id",
    from_=alt.LookupData(df_filtered, "numeric_id", ["noc", "season", "Total_Medals"])
).transform_filter(
    alt.datum.season == season_param
).properties(
    width=800,
    height=400,
    title="Olympic Medals by Country"
).project(
    type="equirectangular"
)

# Add the parameter to the chart
final_chart = world_map.add_params(season_param)

# Display the chart
final_chart


alt.Chart(...)